**点云聚类**

In [4]:
import numpy as np
from plyfile import PlyData
import json
from pathlib import Path

# 读取PLY文件
ply_path = r"/Users/forever/Documents/Code/ScanNetMarker/scene0000_00/scene0000_00_vh_clean.ply"
plydata = PlyData.read(ply_path)

# 获取顶点数据
vertex_data = plydata['vertex']

# 获取点坐标
points = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T
rgbs = np.vstack([vertex_data['red'], vertex_data['green'], vertex_data['blue']]).T

# 获取分割json文件
seg_path = r"/Users/forever/Documents/Code/ScanNetMarker/scene0000_00/scene0000_00_vh_clean.segs.json"
with open(seg_path, 'r') as f:
    seg_data = json.load(f)


# 获取segIndices字段
seg_indices = seg_data['segIndices']

# 根据seg_indices聚合点云
points_by_seg = {}
for i, seg_index in enumerate(seg_indices):
    if seg_index not in points_by_seg:
        points_by_seg[seg_index] = []
    points_by_seg[seg_index].append(points[i])

# # 打印每个seg_index对应的点云数量
# for seg_index, points_list in points_by_seg.items():
#     print(f"seg_index: {seg_index}, 点云数量: {len(points_list)}")

seg_to_obj = {}

# 获取seg 聚合json
seg_path = r"/Users/forever/Documents/Code/ScanNetMarker/scene0000_00/scene0000_00_vh_clean.aggregation.json"
with open(seg_path, 'r') as f:
    seg_data = json.load(f)

    # 获取所有segGroups
    seg_groups = seg_data['segGroups']
    
    # 用于存储每个objectId对应的点云
    points_by_obj = {}
    
# 遍历每个segGroup
for group in seg_groups:
    obj_id = group['objectId'] 
    segments = group['segments']
    label = group['label']
    
    # 如果这个objectId还没有对应的点云列表，创建一个
    if obj_id not in points_by_obj:
        points_by_obj[obj_id] = {
            'label': label,
            'points': []
        }
        
    # 收集这个组内所有segments对应的点
    for seg_id in segments:
        if seg_id in points_by_seg:
            points_by_obj[obj_id]['points'].extend(points_by_seg[seg_id])
    
        seg_to_obj[seg_id] = obj_id

# point_id_to_obj_id = {}
# for obj_id, obj_data in points_by_obj.items():
#     for point in obj_data['points']:
#         point_id_to_obj_id[point] = obj_id
    
for obj_id, obj_data in points_by_obj.items():
    print(f"objectId: {obj_id}, label: {obj_data['label']}, 点云数量: {len(obj_data['points'])}")

objectId: 0, label: window, 点云数量: 11543
objectId: 1, label: window, 点云数量: 3972
objectId: 2, label: table, 点云数量: 66082
objectId: 3, label: kitchen counter, 点云数量: 9767
objectId: 4, label: shower, 点云数量: 46188
objectId: 5, label: curtain, 点云数量: 90616
objectId: 6, label: curtain, 点云数量: 31092
objectId: 7, label: desk, 点云数量: 39611
objectId: 8, label: cabinet, 点云数量: 86274
objectId: 9, label: floor, 点云数量: 319184
objectId: 10, label: sink, 点云数量: 7520
objectId: 11, label: scale, 点云数量: 1534
objectId: 12, label: trash can, 点云数量: 4067
objectId: 13, label: trash can, 点云数量: 6138
objectId: 14, label: trash can, 点云数量: 3435
objectId: 15, label: tv, 点云数量: 9219
objectId: 16, label: pillow, 点云数量: 6567
objectId: 17, label: clock, 点云数量: 1451
objectId: 18, label: backpack, 点云数量: 5277
objectId: 19, label: wall, 点云数量: 65212
objectId: 20, label: wall, 点云数量: 35762
objectId: 21, label: wall, 点云数量: 53276
objectId: 22, label: wall, 点云数量: 19131
objectId: 23, label: wall, 点云数量: 44504
objectId: 24, label: wall, 点云数量: 35

**读取相机参数**

In [6]:
import numpy as np

folder_path = Path(r"/Users/forever/Documents/Code/ScanNetMarker/scene0000_00/images")

img_path = folder_path / "color"

pose_path = folder_path / "pose"

intr_path = folder_path / "intrinsic" / "intrinsic_color.txt"

pose_path = folder_path / "pose"

def read_camera_matrix(intr_path):
    """
    读取相机内参矩阵文件并返回矩阵数据
    Args:
        intr_path: 内参矩阵文件路径
    Returns:
        intr_matrix: 内参矩阵数据列表
    """
    with open(intr_path, "r") as f:
        intr = f.readlines()

        # 将内参矩阵数据转换为浮点数列表
        intr_matrix = []
        for line in intr:
            # 分割每行数据并转换为浮点数
            row = [float(x) for x in line.strip().split()]
            intr_matrix.append(row)
        
    # 将内参矩阵转换为numpy数组
    intr_matrix = np.array(intr_matrix)
    return intr_matrix

intr_matrix = read_camera_matrix(intr_path)

width = int(intr_matrix[0][2])
height = int(intr_matrix[1][2])
print(width, height)

646 489


**使用Open3D获取点的遮挡关系**


In [11]:
import open3d as o3d
import numpy as np
import json
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering
from open3d.visualization.rendering import Open3DScene

    # 初始化GUI应用
gui.Application.instance.initialize()

# 创建窗口
window = gui.Application.instance.create_window("顶点渲染查看器", 1024, 750)

# 创建场景
scene = gui.SceneWidget()
scene.scene = rendering.Open3DScene(window.renderer)
window.add_child(scene)

# 读取网格模型（而不是点云）
mesh = o3d.io.read_triangle_mesh(r"/Users/forever/Documents/Code/ScanNetMarker/scene0000_00/scene0000_00_vh_clean.ply", print_progress=True)

# 计算顶点法线（如果需要）
mesh.compute_vertex_normals()

# 创建材质
mat = rendering.MaterialRecord()
mat.shader = 'defaultLit'  # 使用默认光照着色器
mat.base_color = [0.9, 0.9, 0.9, 1.0]  # 深灰色基础颜色
mat.base_metallic = 0.1  # 非金属
mat.base_roughness = 0.5  # 中等粗糙度
mat.point_size = 3.0  # 点大小
mat.absorption_distance = 0.5
mat.transmission = 0.5
mat.thickness = 1.0

# 添加网格到场景
scene.scene.add_geometry("mesh", mesh, mat)

# 关闭光照
scene.scene.set_lighting(Open3DScene.LightingProfile.NO_SHADOWS, (0, 0, 0))

# 设置背景颜色
scene.scene.set_background(np.array([0, 0, 0, 1]))

# 设置相机视角
bounds = mesh.get_axis_aligned_bounding_box()
scene.setup_camera(60, bounds, bounds.get_center())

# 获取相机控制器
camera = scene.scene.camera
# 设置近裁剪平面和远裁剪平面
# 使用相机内参矩阵设置投影
intrinsics = np.array([[1000, 0, 512],  # fx, 0, cx
                        [0, 1000, 375],    # 0, fy, cy  
                        [0, 0, 1]])        # 0, 0, 1
camera.set_projection(intrinsics, 1, 1000.0, 1024, 750)

gui.Application.instance.run()

Reading PLY file: /Users/forever/Documents/Code/ScanNetMarker/scene0000_00/scene0000_00_vh_clean.ply[========================================] 100%


2025-04-28 01:29:26.126 Python[7741:57918805] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-28 01:29:26.126 Python[7741:57918805] +[IMKInputSession subclass]: chose IMKInputSession_Modern
